In [1]:
## personal imports
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime
from fund.fund import Fund
from pricer.pricer import Pricer as pricer_list
from ranker.ranker import Ranker as ranker_list
from classifier.classifier import Classifier as classifier_list
from portfolio.aportfolio import APortfolio
from parameters.parameters import Parameters as params
## analysis imports
from analysis.quarterly_analysis import QuarterlyAnalysis as quarterly_analysis
from analysis.weekly_analysis import WeeklyAnalysis as weekly_analysis

## risk and return class imports
from returns.products import Products as returns_products_class
from fund.fund import Fund

## standard imports
from statistics import variance
import math
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
backtest_start_date = datetime(2003,1,1)
backtest_end_date  = datetime(2023,1,1)
current_start_date = datetime(2023,1,1)

In [3]:
market = Market()

In [4]:
portfolio = APortfolio(pricer_list.WEEKLY_STOCK_SPECULATION
                          ,classifier_list.NONE
                          ,ranker_list.NONE)
portfolio_ii = APortfolio(pricer_list.WEEKLY_STOCK_ROLLING
                           ,classifier_list.NONE
                          ,ranker_list.NONE)
portfolio_iii = APortfolio(pricer_list.WEEKLY_STOCK_WINDOW
                          ,classifier_list.NONE
                          ,ranker_list.NONE)
portfolio_iv = APortfolio(pricer_list.WEEKLY_CRYPTO_ROLLING
                          ,classifier_list.NONE
                          ,ranker_list.NONE)
portfolio_v = APortfolio(pricer_list.WEEKLY_CRYPTO_WINDOW
                          ,classifier_list.NONE
                          ,ranker_list.NONE)
portfolio_vi = APortfolio(pricer_list.WEEKLY_CRYPTO_SPECULATION
                          ,classifier_list.NONE
                          ,ranker_list.NONE)

portfolios = []
portfolios.append(portfolio)
portfolios.append(portfolio_ii)
portfolios.append(portfolio_iii)
portfolios.append(portfolio_iv)
portfolios.append(portfolio_v)
portfolios.append(portfolio_vi)

In [5]:
fund = Fund(portfolios,backtest_start_date,backtest_end_date,current_start_date)
market = Market()
fund.initialize_portfolios()

In [6]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [7]:
market.connect()
tyields = returns_products_class.tyields(market.retrieve("tyields"))
bench_returns = returns_products_class.spy_bench(market.retrieve("spy"))
market.disconnect()

In [ ]:
analysis = []
parameters = params.parameters()
indexer = list(parameters[0].keys()) + ['portfolio']
indexer
for portfolio in tqdm(portfolios):
    portfolio.db.connect()
    for parameter in tqdm(parameters):
        try:
            trade = portfolio.db.query("trades",parameter)
            if trade.index.size > 0:
                indexer = list(parameters[0].keys()) + ['portfolio']
                positions = int(portfolio.pricer_class.positions / 5) if portfolio.pricer_class.asset_class.value == "stocks" else 1
                pricer_timeframe =  portfolio.pricer_class.time_horizon_class.naming_convention
                trade = trade[trade["position"] <= positions]
                ledger = trade.sort_values(["year",pricer_timeframe]).copy()
                anas = weekly_analysis if pricer_timeframe == "week" else quarterly_analysis
                if ledger.index.size > 0:
                    cumulative = anas.trade_analysis(ledger,positions,tyields,bench_returns)
                    for key in parameter.keys():
                        cumulative[key] = parameter[key]
                    cumulative["portfolio"] = portfolio.name
                    analysis.append(cumulative[indexer + ["pv","beta","rrr","sharpe"]].iloc[-1])
        except Exception as e:
            print(str(e))
            continue
    portfolio.db.disconnect()
a = pd.DataFrame([x.to_dict() for x in analysis])

 77%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                     | 37/48 [01:02<00:10,  1.09it/s]


 54%|█████████████████████████████████████████████████████████████████████████████████████████▍                                                                           | 26/48 [03:49<03:22,  9.22s/it]


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 47/48 [06:10<00:07,  7.48s/it]

In [ ]:
a.sort_values("pv",ascending=False).head(20)

In [ ]:
a.sort_values("pv",ascending=False).groupby("portfolio").nth(0).reset_index()

In [ ]:
top = a.sort_values("pv",ascending=False).iloc[0]
db = ADatabase(top["portfolio"])
parameter = top[[x for x in indexer if x != "portfolio"]].to_dict()
parameter
db.connect()
analysis = db.query("trades",parameter)
db.disconnect()
analysis.head()

In [ ]:
anas = weekly_analysis
naming = "week"
positions = int(int(analysis["position"].max())/5)
ledger = analysis[["year",naming,"ticker",f"{naming}ly_delta",f"{naming}ly_delta_sign","actual_returns","position"]]
ledger = ledger[ledger["year"]>=backtest_start_date.year]
ledger = ledger.merge(sp500[["ticker","GICS Sector","Security"]],on="ticker",how="left")

In [ ]:
ledger

In [ ]:
plt.figure(figsize=(12, 7), dpi=80)
plt.scatter(ledger[f"{naming}ly_delta"] * 100, (ledger["actual_returns"]-1)*100,s=10)

In [ ]:
ledger.sort_values(["year",naming,"position"],ascending=True).head(20)

In [ ]:
ledger["actual_returns"].mean()

In [ ]:
ledger[ledger["actual_returns"]>1].index.size / ledger.index.size

In [ ]:
portfolio = ledger.pivot_table(index=["year",naming],columns="position",values="actual_returns").fillna(1).reset_index()

In [ ]:
positions

In [ ]:
cumulative = anas.iteration_analysis(portfolio,positions,bench_returns)

In [ ]:
cumulative.head(10)

In [ ]:
plt.figure(figsize=(12, 7), dpi=80)
plt.plot(cumulative["date"],cumulative["pv"])
plt.plot(cumulative["date"],cumulative["bench"])
plt.legend(["pv","bench"])

In [ ]:
cumulative.tail(20)

In [ ]:
plt.figure(figsize=(12, 7), dpi=80)
position_columns = [x for x in cumulative.columns if x not in ["year","week","quarter","date_string","date","pv","adjclose","bench"]]
for i in position_columns:
    position = cumulative[i]
    plt.plot(cumulative["date"],position)

plt.show()


In [ ]:
ledger.groupby(["GICS Sector"]).mean().sort_values("actual_returns",ascending=False)

In [ ]:
ledger.groupby(["year",naming,"ticker","Security"]).prod().sort_values("actual_returns",ascending=False).head(30)

In [ ]:
industry_analysis = ledger.pivot_table(index=["year",naming],columns="GICS Sector",values="actual_returns").fillna(1).reset_index()
industry_analysis[[x for x in industry_analysis.columns if x != "sell_date"]].cumprod()